# Practical Session - Final

Student(s): Aubery Cléa - Shinawi Aymeric

## 1. Problem selection

Choose one of the two following problems (delete the other cell).
You can find extra information regarding both problems and their associated models in
the main course.

### Problem 1: Single-Machine Scheduling

A machine needs to produce a set L of n items. Each
item i has a manufacturing time pi, a release time ri (time at which its components are
available) and a deadline di (time at which the time should be manufactured). The
factory can only produce one item at a time, and manufacturing process cannot be
interrupted (non-preemptive machine), what is the schedule that minimize the
overdue deliveries?

**Variables:**

- $x_{ij}$ - Binary variable indicating if $i$ is manufactured before $j$.
- $s_{i}$ - Continuous variable indicating the starting time of task $i$.
- $y_{i}$ - Binary variable indicating if the item $i$ is overdue.

**Model:**

$$
    \begin{align}
      \text{min.} \quad & \sum_{j=1}^{n} y_{i}                 &                                           \\
      \text{s.t.} \quad & s_j \geq s_i + p_i - M^1_{ij} (1 - x_{ij}), & \forall i,j \in L,\ i \ne j \\
                        & s_i + p_i \leq d_i + M^2_{i} y_i,          & \forall i \in L             \\
                        & x_{ij} + x_{ji} = 1,                 & \forall i,j \in L,\ i \ne j \\
                        & x_{ij} \in \left\{0,~1\right\},      & \forall i,j \in L                         \\
                        & y_{i} \in \left\{0,~1\right\},       & \forall i \in L                           \\
                        & s_{i} \geq r_{i},                    & \forall i \in L
    \end{align}
$$

## 2. Preparation

### 2.1. Instance

Implement a class to hold the various data parameters required for the problem (e.g.,
manufacturing time or aircraft target landing time).
Your class can be a simple data holder with various fields or can contain extra methods
for later use.

In [1]:
def createItem(pi, ri, di):
    return [pi, ri, di]

class Instance:
    def __init__(self):
      self.items=[]

    def addItems(self, items):
      self.items.extend(items)

    def addItem(self, item):
      self.items.append(item)

    def getItems(self):
      return self.items

    def getNbItems(self):
      return len(self.items)

    def getItem(self, i):
      return self.items[i]

    def getPi(self, i):
      return self.items[i][0]

    def getRi(self, i):
      return self.items[i][1]

    def getDi(self, i):
      return self.items[i][2]

### 2.2. Solution

Implement a to hold a proper solution for the chosen problem.
Your class should not contain the values for all variables in your model but only the
relevant ones.

In [2]:
# need to stock  ?
#    xij - Binary variable indicating if i is manufactured before j.  --> not really, can be deducted
#    yi - Binary variable indicating if the item i is overdue. --> not really, can be deducted
#    si - Continuous variable indicating the starting time of task i.  --> yes

class Solution:
    instance: Instance
    objective: int
    s:[]
    def __init__(self):
      self.s=[]
      self.objective=-1

    def setInstance(self, instance):
      self.instance = instance
      self.s=[-1 for i in range(instance.getNbItems())]

    def setS(self, S):
      if(len(S)==len(self.s)):
        self.s = S
      else:
        raise Exception("Incorrect length for Y!")

    def setSi(self, i,  si):
      self.s[i]=si

    def getS(self):
      return self.s

    def getSi(self, i):
      return self.s[i]

    def getInstance(self):
      return self.instance

    def setObjective(self, obj):
      self.objective = obj
    
    def getObjective(self):
      return self.objective

Implement the function below to check if a solution is valid or not.

In [3]:
def check_solution(solution: Solution) -> bool:
    instance = solution.getInstance()
    for i in range(instance.getNbItems()):
        # item  : ri, pi, di :  release time ri, manufacturing time pi and a deadline di
        soli = solution.getSi(i)
        ri = instance.getRi(i) 
            
        # verification : task is executed
        if(soli==-1):
            return False
        # verification : task is started after release time
        elif(ri>soli):
            return False   
    sol = []
    
    # verification : task is started after previous is ended 
    # sort the tasks by starting time
    for index, si in enumerate(solution.getS()):
        sol.append([si, index])
    sol.sort()
    # check appropriate values
    for i in range(instance.getNbItems()-1):
        si_0 = sol[i][0]
        pi_0 = instance.getPi(sol[i][1])
        si_1 = sol[i+1][0]
        if(si_0+pi_0>si_1):
            return False
            
    return True

def check_solution_ontime(solution : Solution) -> bool :
    instance = solution.getInstance()
    res = check_solution(solution)
    for i in range(instance.getNbItems()):
        soli = solution.getSi(i)
        pi = instance.getPi(i)
        di = instance.getDi(i)
        # verification : delay (exec + process > deadline)    
        if pi + soli > (di):
            return False
    return res
    

 <font color='blue'> We added a function that checks if a solution is on time or not.</font>

### 2.3. Data Generation

Propose, explain and implement a method to generate instances of various sizes (e.g.,
number of items or number of aircraft).

<font color='blue'>We generate n items with the values :  
    
- <font color='blue'>$pi$- random between 0 and max task duration
- <font color='blue'>$ri$- random between 0 and max task duration * n
- <font color='blue'> $di$ - random between 0 and max task duration * n + pi + ri
    
<font color='blue'> The max task duration is defined in parameter. The default value is 10.


In [4]:
import random

def generate_instance(size: int, rng: random.Random = random.Random(), maxDuration: int = 10) -> Instance:
    # Generation of the n items. [pi, ri, di]
    # pi : On peut choisir une durée max des tâches en paramètre (défaut : 15)
    # ri : On choisi de façon aléatoire entre 0 et (pi-1)*size
    # di : On choisi de façon aléatoire entre pi et pi*size
    items = [[] for i in range(size)]
    for i in range(size):
        ri = int (rng.random()*size*maxDuration)
        pi = int (rng.random()*maxDuration+1)

        di = int(rng.random()*maxDuration) + pi +ri
        items[i] = createItem(pi, ri, di)

    instance = Instance()
    instance.addItems(items)
    return instance


## 3. Model implementation

### 3.1. Model

Implement the function below that should create an appropriate `docplex` model
given an instance of the chosen problem.

Use appropriate values for the big-$M$ constants in the model.

In [5]:
from docplex.mp.model import Model

def model_for(instance: Instance) -> Model:
    model = Model("Single-Machine Scheduling")
    
    L = instance.getNbItems()
    
    # upper bound for y : max task duration * L + biggest realease time
    M = max([instance.getPi(i) for i in range(L)]) * L + max([instance.getRi(i) for i in range(L)]) * 999

    # Create si, yi
    #s = model.integer_var_list(L, 0, M, name="s_")
    s = model.continuous_var_list(L, 0, M, name=f's_')
    y = model.binary_var_list(L, name="y_")
    
    # Create xi
    x = []
    for i in range(L):
        x.append(model.binary_var_list(L, name=f"x{i}_"))
        
    # Objective contraint
    model.minimize(model.sum(y[i] for i in range(L)))
    
    # --- Constraints ---------
    # sj >= si + pi - M(1, ij)(1-xij)
    for i in range(L):
        for j in range(L):
            if i != j:
                model.add_constraint(s[j] >= s[i] + instance.getItem(i)[0] - M * (1 - x[i][j]))

    # si + pi <= di + M(2, i)(yi)
    for i in range(L):
        model.add_constraint_(s[i] + instance.getPi(i) <= instance.getDi(i) + M * y[i])
      
    for i in range(L):
        # xij + xji = 1
        for j in range(i):
            model.add_constraint_(x[i][j]+x[j][i]==1)          
    for i in range(L):    
        # si >= ri    
        model.add_constraint_(s[i]>=instance.getRi(i))
               
    # -------------------------
        w
    return model
    

### 3.2. Resolution

Complete the function below to construct a proper `Solution` from the obtained
`docplex` solution.

In [6]:
def solve(instance: Instance) -> Solution | None:
    model = model_for(instance)

    # enable logging
    model.log_output = True

    # solve
    solution = model.solve()
    
    
    if not solution:
        return None

    # TODO: convert the docplex solution to an appropriate Solution
    print("---------------")
    sol = Solution()
    sol.setInstance(instance)
    sol.setObjective(solution.get_objective_value())
    
    for i in range(instance.getNbItems()):
        print(solution.get_value(f"y__{i}"))
        sol.setSi(i, solution.get_value(f"s__{i}"))
  
    return sol

In [7]:
def test_instance(instance):
    solution_generated = solve(instance)
    print(instance.getItems())
    print("S = ", solution_generated.getS())
    print("obj = ", solution_generated.getObjective())
    print("\n")

    #pi, ri, di
    print(check_solution(solution_generated))
    print(check_solution_ontime(solution_generated))

In [8]:
test_instance(generate_instance(3))

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 3 rows and 3 columns.
MIP Presolve modified 15 coefficients.
Aggregator did 3 substitutions.
Reduced MIP has 9 rows, 9 columns, and 24 nonzeros.
Reduced MIP has 6 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Found incumbent of value 3.000000 after 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 3 coefficients.
Reduced MIP has 9 rows, 9 columns, and 24 nonzeros.
Reduced MIP has 6 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Clique table members: 3.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.00 sec. (0.02 

## 4. Evaluation

### 4.1. Simple evaluation

Evaluate the model and solution implemented above on various instances generated by
your method.

What size of instances can your model solve? Which parameters have the biggest impact
on the resolution time?

In [9]:
import time

sizes = [*range(10, 162, 10)]

instances = []
solutions = []
exec_times = []

for i in sizes:
    instance = generate_instance(i)
    
    time1 = time.time()  
    solution_generated = solve(instance)    
    time2 = time.time()  
    
    instances.append(instance)
    solutions.append(solution_generated)
    exec_times.append(time2-time1)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 10 rows and 10 columns.
MIP Presolve modified 190 coefficients.
Aggregator did 45 substitutions.
Reduced MIP has 100 rows, 65 columns, and 290 nonzeros.
Reduced MIP has 55 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.30 ticks)
Found incumbent of value 10.000000 after 0.00 sec. (0.39 ticks)
Probing time = 0.00 sec. (0.09 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 11 coefficients.
Reduced MIP has 100 rows, 65 columns, and 290 nonzeros.
Reduced MIP has 55 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.19 ticks)
Probing time = 0.00 sec. (0.09 ticks)
Clique table members: 44.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time =

      0     0       -0.0000    15       33.0000        0.0000      302  100.00%
*     0+    0                            8.0000        0.0000           100.00%
Detecting symmetries...
      0     2       -0.0000    15        8.0000        0.0000      302  100.00%
Elapsed time = 0.07 sec. (50.74 ticks, tree = 0.02 MB, solutions = 3)
*   226    79      integral     0        7.0000        0.0000      737  100.00%
*   445   126      integral     0        5.0000        1.0000     1198   80.00%
*  1130   210      integral     0        4.0000        3.0000     3814   25.00%
*  1273   227      integral     0        4.0000        3.0000     4045   25.00%
*  1543   184      integral     0        4.0000        3.0000     6419   25.00%

Cover cuts applied:  29

Root node processing (before b&c):
  Real time             =    0.06 sec. (50.22 ticks)
Parallel b&c, 12 threads:
  Real time             =    0.15 sec. (115.64 ticks)
  Sync time (average)   =    0.05 sec.
  Wait time (average)   =    0.00

*     0+    0                           70.0000        0.0000           100.00%
*     0+    0                           68.0000        0.0000           100.00%
      0     0       -0.0000    14       68.0000        0.0000     1106  100.00%
Detecting symmetries...
      0     2       -0.0000    14       68.0000        0.0000     1106  100.00%
Elapsed time = 0.13 sec. (189.16 ticks, tree = 0.02 MB, solutions = 2)
*    27+    8                           17.0000        0.0000           100.00%
*    80+   32                           16.0000        0.0000           100.00%
*   352   173      integral     0       12.0000        0.0000     2132  100.00%
*   389   194      integral     0       10.0000        0.0000     2269  100.00%
*   530   182      integral     0        9.0000        0.0000     2834  100.00%
*   676   183      integral     0        7.0000        0.0000     3039  100.00%
*   779   220      integral     0        5.0000        0.0000     3605  100.00%
*  1143   250      integr

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 100 rows and 100 columns.
MIP Presolve modified 19778 coefficients.
Aggregator did 4950 substitutions.
Reduced MIP has 10000 rows, 5150 columns, and 29900 nonzeros.
Reduced MIP has 5050 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (25.28 ticks)
Found incumbent of value 100.000000 after 0.04 sec. (47.73 ticks)
Probing time = 0.00 sec. (4.60 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 141 coefficients.
Reduced MIP has 10000 rows, 5150 columns, and 29900 nonzeros.
Reduced MIP has 5050 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (15.94 ticks)
Probing time = 0.01 sec. (4.53 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time 

Parallel b&c, 12 threads:
  Real time             =    0.71 sec. (405.76 ticks)
  Sync time (average)   =    0.24 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    1.17 sec. (1355.38 ticks)
---------------
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1.94512303417527e-16
0
0
0
0
0
0
0
0
0
0
0
0
0
2.5063012590822076e-06
5.84850044438312e-06
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1.672468739468769e-06
0
0
0
0
0
0
0
0
0
0
5.8535231101272634e-06
0
0
0
0
0
8.357633151721659e-07
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7.526002338078077e-06
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 130 rows and 130 columns.
MIP Presolve modified 33154 coefficients.
Aggregator did 8385 substitutions.
Reduced MIP has 16900 rows, 8645 columns, and 50570 nonzeros.
Reduced MIP has 8515 binaries, 

Reduced MIP has 12880 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (64.27 ticks)
Found incumbent of value 160.000000 after 0.10 sec. (114.33 ticks)
Probing time = 0.01 sec. (7.42 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 90 coefficients.
Reduced MIP has 25600 rows, 13040 columns, and 76640 nonzeros.
Reduced MIP has 12880 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.04 sec. (41.20 ticks)
Probing time = 0.01 sec. (7.35 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.16 sec. (225.28 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                          160.0000        0.0000           100.00%
*     0+    0                          159.0000        0.0000        

In [10]:
print(sizes)
print(exec_times)

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160]
[0.053511857986450195, 0.07059335708618164, 0.17435908317565918, 0.3321542739868164, 0.9210841655731201, 0.5346486568450928, 1.31673002243042, 0.9836273193359375, 1.0401287078857422, 1.8412401676177979, 1.5912468433380127, 2.2217509746551514, 2.0834851264953613, 1.5912866592407227, 2.0970458984375, 2.2935421466827393]


<font color='blue'> Above we displayed the exection time corresponding to different model sizes.
    
<font color='blue'> With a instance size of 100, the model is still fast (1.2 second). For 150 values, we have an execution time of 2 seconds, which is still quite short.
    
    
<font color='blue'> The biggest impact on performance is the number of possible values that can be taken by the variables. Our way of generating the data may have big constraints wich could explain why there is not long execution times. 
   

### 4.2. Relaxation

Evaluate the impact of having tighter big-$M$ values in your model to the appropriate
relaxation.

**Tips:**

- Adapt the `model_for` function (or create a new one) to easily change the big-$M$
  values.
- You can use the following code to solve the relaxation of your model:

```python
from docplex.mp.relax_linear import LinearRelaxer

lp = LinearRelaxer.make_relaxed_model(model)
solution_relaxed = lp.solve(log_output=True)
```

In [11]:
def model_for_relaxation(instance: Instance, M: int = -1) -> Model:
    model = Model("Single-Machine Scheduling")
    
    L = instance.getNbItems()
    
    # upper bound for y : temps max * L + biggest realease time
    if M < 0:
        M = max([instance.getPi(i) for i in range(L)]) * L + max([instance.getRi(i) for i in range(L)]) * 999

    # Create si, yi
    #s = model.integer_var_list(L, 0, M, name="s_")
    s = model.continuous_var_list(L, 0, M, name=f's_')
    y = model.binary_var_list(L, name="y_")
    
    # Create xi
    x = []
    for i in range(L):
        x.append(model.binary_var_list(L, name=f"x{i}_"))
        
    # Objective contraint
    model.minimize(model.sum(y[i] for i in range(L)))
    
    # --- Constraints ---------
    # sj >= si + pi - M(1, ij)(1-xij)
    for i in range(L):
        for j in range(L):
            if i != j:
                model.add_constraint(s[j] >= s[i] + instance.getItem(i)[0] - M * (1 - x[i][j]))

    # si + pi <= di + M(2, i)(yi)
    for i in range(L):
        model.add_constraint_(s[i] + instance.getPi(i) <= instance.getDi(i) + M * y[i])
      
    for i in range(L):
        # xij + xji = 1
        for j in range(i):
            model.add_constraint_(x[i][j]+x[j][i]==1)          
    for i in range(L):    
        # si >= ri    
        model.add_constraint_(s[i]>=instance.getRi(i))
      
    # -------------------------
        
    return model

In [12]:
from docplex.mp.relax_linear import LinearRelaxer

def solve_relaxation(instance: Instance, M:int =-1) -> Solution | None:
    model = model_for_relaxation(instance , M=M)
    lp = LinearRelaxer.make_relaxed_model(model)
    # enable logging
    lp.log_output = True

    # solve
    solution = lp.solve()
    
    if not solution:
        return None

    # TODO: convert the docplex solution to an appropriate Solution
    print("---------------")
    sol = Solution()
    sol.setInstance(instance)
    sol.setObjective(solution.get_objective_value())
    
    for i in range(instance.getNbItems()):
        print(solution.get_value(f"y__{i}"))
        sol.setSi(i, solution.get_value(f"s__{i}"))
  
    return sol

In [13]:
solution = solve_relaxation(generate_instance(10))
print(solution.getS())

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 12 rows and 12 columns.
Aggregator did 45 substitutions.
Reduced LP has 98 rows, 63 columns, and 286 nonzeros.
Presolve time = 0.00 sec. (0.14 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =             0.000000
---------------
0
0
0
0
0
0
0
0
0
0
[73.0, 84.0, 93.0, 95.0, 14.0, 81.0, 25.0, 85.0, 93.0, 90.0]


In [14]:
Ms = [-1] +  [*range(2, 10, 1)] + [*range(10, 41, 3)] + [100**i for i in range(1, 6)]
instances = []
solutions = []
exec_times = []

for m in Ms:
    time1 = time.time()
    instance = generate_instance(100, maxDuration=10)
    solution = solve_relaxation(instance, m)    
    time2 = time.time()
    
    exec_times.append(time2-time1)
    
    instances.append(instance)
    solutions.append(solution_generated)


Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 12 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 10 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 112 rows and 112 columns.
Aggregator did 4950 substitutions.
Reduced LP has 9988 rows, 5138 columns, and 29876 nonzeros.
Presolve time = 0.01 sec. (14.88 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =             0.000000
Perturbation started.
Iteration:   101   Dual objective     =             0.000000
Iteration:   419   Dual objective     =             0.000000
Iteration:  1008   Dual objective     =             0.000000
Iteration:  1415   Dual objective     =             0.000000
Iteration:  1804   Dual objective     =             0.000000
Iteration:  2229   Dual objective     =             0.000000
Iteration:  284

CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 12 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 10 threads...
 * Starting primal Simplex on 1 thread...
Bound infeasibility column 's__0'.
Presolve time = 0.00 sec. (1.36 ticks)
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 12 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 10 threads...
 * Starting primal Simplex on 1 thread...
Bound infeasibility column 's__0'.
Presolve time = 0.00 sec. (1.36 ticks)
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 12 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 10 threads...
 * Starting primal Simplex on 

Iteration:   622   Dual objective     =             0.000000
Iteration:   625   Dual objective     =             0.000000
Iteration:   629   Dual objective     =             0.000000
Iteration:   639   Dual objective     =             0.000000
Iteration:   657   Dual objective     =             0.000000
Iteration:   661   Dual objective     =             0.000000
Iteration:   676   Dual objective     =             0.000000
Iteration:   687   Dual objective     =             0.000000
Iteration:   712   Dual objective     =             0.000000
Iteration:   732   Dual objective     =             0.000000
Iteration:   743   Dual objective     =             0.000000
Iteration:   745   Dual objective     =             0.000000
Iteration:   761   Dual objective     =             0.000000
Iteration:   775   Dual objective     =             0.000000
Iteration:   779   Dual objective     =             0.000000
Removing perturbation.

Barrier solved model.

---------------
0
0
0
0
0
0
0
0
0
0
0


In [15]:
print(Ms)
print(exec_times)

[-1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 100, 10000, 1000000, 100000000, 10000000000]
[1.4183459281921387, 1.3090295791625977, 1.3479259014129639, 1.552210807800293, 1.5041460990905762, 1.3333656787872314, 1.3463683128356934, 1.3874599933624268, 1.37025785446167, 1.4810385704040527, 1.3735711574554443, 1.3685288429260254, 1.4091055393218994, 1.449810266494751, 1.3724732398986816, 1.3833906650543213, 1.562969446182251, 1.4114158153533936, 1.345782995223999, 1.3429358005523682, 1.3906810283660889, 1.4340581893920898, 1.4344596862792969, 1.500225305557251, 1.5355935096740723]


<font color='blue'> Above the different M values we tested, and the corresponding execution time.  
    
    

<font color='blue'> 
 We see here that the relaxed version doesn't change the execution time much.

### 4.3. Visualization (Bonus)

Propose a method to visualize the instance and solution obtained by your model using
library such as `matplotlib`.

In [16]:
def visualization(instance: Instance, solution:Solution):
    current = 0
    
    sol = []
    for index, si in enumerate(solution.getS()):
        sol.append([si, index])
    sol.sort()
    
    for si, index in sol:
        pi = instance.getPi(index)
        wait_time = abs(current-si)
        print("-"*int(wait_time), end="")
        print((str)(index)*pi, end="")
        current = pi+si

In [17]:
instance = generate_instance(4)
sol = Solution()
sol.setInstance(instance)
sol.setS([2, 8, 16, 7])
visualization(instance, sol)

--000003333333333---------1111111122